
**Install requirements**

In [0]:
!pip3 install 'Pillow==6.1'
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
#!pip3 install --upgrade 'pillow'

**Import libraries**

In [0]:
import os
import logging
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

from PIL import Image
import itertools

import sys
import io
import copy 

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

from torch.autograd import Function
import torchvision
from torchvision import transforms

from torchvision.models import alexnet
from torchvision.datasets import VisionDataset


**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 7 

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using DSG
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
NUM_EPOCH = 30       # Total number of training epochs (iterations over dataset)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LOG_FREQUENCY = 5

**Define Data Preprocessing**

In [0]:
# Define transforms for training phase
transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) 
])


Dataset

In [0]:
#Funzioni per il caricamento del dataset

def pil_loader(path):
  # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
  img = Image.open(open(path, 'rb'), mode='r')
  return img.convert('RGB')

def make_dataset(dir, class_to_idx, extensions=None, is_valid_file=None):
  images = []
  dir = os.path.expanduser(dir)
  if not ((extensions is None) ^ (is_valid_file is None)):
      raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")
  if extensions is not None:
      def is_valid_file(x):
          return has_file_allowed_extension(x, extensions)
  for target in sorted(class_to_idx.keys()):
      d = os.path.join(dir, target)
      if not os.path.isdir(d):
          continue
      for root, _, fnames in sorted(os.walk(d, followlinks=True)):
          for fname in sorted(fnames):
              path = os.path.join(root, fname)
              if is_valid_file(path):
                  item = (path, class_to_idx[target])
                  images.append(item)

  return images

def has_file_allowed_extension(filename, extensions):
  """Checks if a file is an allowed extension.
  Args:
      filename (string): path to a file
      extensions (tuple of strings): extensions to consider (lowercase)
  Returns:
      bool: True if the filename ends with one of given extensions
  """
  return filename.lower().endswith(extensions)

def default_loader(path):
  from torchvision import get_image_backend
  if get_image_backend() == 'accimage':
      return accimage_loader(path)
  else:
      return pil_loader(path)


class PACS(VisionDataset):

    def __init__(self, root, folds_to_use, split='train', transform=None, target_transform=None):

        super(PACS, self).__init__(root, transform=transform, target_transform=target_transform)

        IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

        aux_root = self.root + "/" + folds_to_use[0]
        classes, class_to_idx = self._find_classes(aux_root)

        samples = []
        self.domains = []

        for fold in folds_to_use:
          aux_root = self.root + "/" + fold
          aux_samples_tuple = make_dataset(aux_root, class_to_idx, IMG_EXTENSIONS)
          for s in aux_samples_tuple:
            self.domains.append(fold)
          samples += aux_samples_tuple

         
        if len(samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(IMG_EXTENSIONS)))
            
           
        self.loader =  default_loader
        self.extensions = IMG_EXTENSIONS

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]
        self.transform = transform
        self.target_transform = target_transform


    def _find_classes(self, dir):
        if sys.version_info >= (3, 5):
            # Faster and available in Python 3.5 and above
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()

        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx

    def __getitem__(self, index):
   # Provide a way to access image and label via index. Image should be a PIL Image  label can be int
      path, target = self.samples[index]
      sample = self.loader(path)
      if self.transform is not None:
          sample = self.transform(sample)
      if self.target_transform is not None:
          target = self.target_transform(target)

      return sample, target

    def __len__(self):
        length = len # Provide a way to get the length (number of elements) of the dataset
        return len(self.samples)
        

DANN_AlexNet

In [0]:
__all__ = ['DANN_AlexNet', 'DANN_alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

class ReverseLayerF(Function):
    # Forwards identity
    # Sends backward reversed gradients
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
      output = grad_output.neg() * ctx.alpha
      return output, None

class DANN_AlexNet(nn.Module):

  def __init__(self, num_classes=1000):
      super(DANN_AlexNet, self).__init__()

      self.features = nn.Sequential(
          nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=3, stride=2),
          nn.Conv2d(64, 192, kernel_size=5, padding=2),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=3, stride=2),
          nn.Conv2d(192, 384, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(384, 256, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(256, 256, kernel_size=3, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=3, stride=2),
      )
      self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

      self.classifier = nn.Sequential(
          nn.Dropout(),
          nn.Linear(256 * 6 * 6, 4096),
          nn.ReLU(inplace=True),
          nn.Dropout(),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, num_classes),
      )
      self.domain_classifier = nn.Sequential(
          nn.Dropout(),
          nn.Linear(256 * 6 * 6, 4096),
          nn.ReLU(inplace=True),
          nn.Dropout(),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, 2),
      )

  def forward(self, x, alpha=None):

      x = self.features(x)
      x = self.avgpool(x)
      x = torch.flatten(x, 1)
      if alpha is not None:
        reverse_x = ReverseLayerF.apply(x, alpha)
        return self.domain_classifier(reverse_x)
      else:
        return self.classifier(x)

def DANN_alexnet(pretrained=False, progress=True, **kwargs):

    model = DANN_AlexNet(**kwargs)
    if pretrained:
        Alex_state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        state_dict = model.state_dict() 

        for key, values in Alex_state_dict.items():
          state_dict[key] = values 
          if key[:10] == "classifier":
            aux_str = "classifier" + key[10:]
            state_dict[aux_str] = values

        model.load_state_dict(state_dict)
        
    model.classifier[6] = nn.Linear(4096, NUM_CLASSES)

    return model
    

Utility

In [0]:
'''Class used to print the details on the trend of the losses during the execution
for the point 4. The method print_log reports the various losses during the
phases of the execution while the method print_loss prints the losses in an
easily usable format to print graphics on Python.

If you want to see the values of the loss remove comments from the last two
methods of the code that implements point 4A-B and point 4C-D.'''

class Log:

  def __init__(self, str):
    self.log = [["str", str]]

  def add_iper(self, list):
    self.log.append(["iper", list])

  def add_epoch(self, list):
    self.log.append(["epoch", list])

  def add_str(self, list):
    self.log.append(["str", list])

  def add_loss(self, list):
    self.log.append(["loss", list])

  def concLOG(self, other_Log):
    self.log.append(["new log", ""])
    self.add_str("\n-------------------------------------------------------------\n")
    self.log += other_Log.log

  def print_log(self):

    for ele in self.log:

      if ele [0] == "iper":
        
        if len(ele[1]) == 2:
          aux_str = "Inizio stampa dati per: lr = " + str(ele[1][0]) + " step_size = " +  str(ele[1][1])  + ".\n"
        else:
          aux_str = "Inizio stampa dati per: lr = " + str(ele[1][0]) + " step_size = " +  str(ele[1][1])  +  " alpha = " + str(ele[1][2]) + ".\n"
        print(aux_str)

      elif ele[0] == "epoch":
        print ("Starting epoch: " + str(ele[1][0]) + " / " + str(str(ele[1][1])) + " lr = " + str(ele[1][2]) )
      
      elif ele[0] == "loss":
        aux_str = ""
        firts = True

        for index in range(len(ele[1])):
          if firts:
            firts = False
            aux_str += "  Loss per fase " + str(index) + ": " + str(ele[1][index])
          else:
            aux_str += "\n  Loss per fase " + str(index) + ": " + str(ele[1][index])
        
        print(aux_str)
      
      else:
        print(ele[1])

  def print_loss(self):

    str_aux = "\n"
    loss1 = ""
    loss2 = ""
    first = True
    DANN = False
    last = False
    first_block = True
    first_ele = True

    for ele in self.log:

      if ele[0] == "loss":
        if first:
          str_aux += "["
          first = False
          last = True
        
        elif len(ele[1]) == 1 :
          str_aux += ", "

        if len(ele[1]) == 1:
          str_aux += str(ele[1][0])

        else:
          DANN = True
          if first_ele:
            loss1 += str(ele[1][0])
            loss2 += str(ele[1][1])
            first_ele = False
          else:
            loss1 += ", " + str(ele[1][0])
            loss2 += ", " + str(ele[1][1])
      
      elif ele[0] != "epoch":
       
        if DANN :
          str_aux += loss1 + "], ["
          str_aux += loss2 + "]"
          first_ele = True
          loss1 = ""
          loss2 = ""
          DANN = False

        if last:
          str_aux += "]"
          first = True
          last = False

        if ele[0] == "new log":
          if first_block:
            str_aux += "["
            first_block = False
          else:
            str_aux += "]\n["





    str_aux += "]]"
    print(str_aux)
 
def prepareDataloader(transform, source_domains, target_domains, val1_domains, val2_domains):
  # Clone github repository with data
  if not os.path.isdir('./Homework3-PACS'):
    !git clone https://github.com/MachineLearning2020/Homework3-PACS

  DATA_DIR = 'Homework3-PACS/PACS'

  # Prepare Pytorch Datasets
  domains = source_domains + target_domains + val1_domains + val2_domains


  dataset = PACS(DATA_DIR, domains, transform=transform)

  source_indexes = []
  target_indexes = []
  val1_indexes = []
  val2_indexes = []



  for index in range(len(dataset)):
    if dataset.domains[index] in source_domains: 
      source_indexes.append(index)
    elif dataset.domains[index] in target_domains: 
      target_indexes.append(index)
    elif dataset.domains[index] in val1_domains: 
      val1_indexes.append(index)
    else:
      val2_indexes.append(index)

  source_dataset = Subset(dataset, source_indexes)
  target_dataset = Subset(dataset, target_indexes)
  val1_dataset = Subset(dataset, val1_indexes)
  val2_dataset = Subset(dataset, val2_indexes)

  # Check dataset sizes
  print('Art Painting Dataset: {}'.format(len(source_dataset)))
  print('Photo Dataset: {}'.format(len(target_dataset)))  
  print('Cartoon Dataset: {}'.format(len(val1_dataset)))  
  print('Sketch Dataset: {}'.format(len(val2_dataset)))  

  # Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
  source_dataloader = DataLoader(source_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
  target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
  val1_dataloader = DataLoader(val1_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
  val2_dataloader = DataLoader(val2_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)


  return source_dataloader, target_dataloader, val1_dataloader, val2_dataloader

def test (net, dataloader):

  net.train(False)
  running_corrects = 0

  for images, labels in dataloader:

    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = net(images)
    _, preds = torch.max(outputs.data, 1)

    running_corrects += torch.sum(preds == labels.data).data.item()
    
  return running_corrects / float(len(dataloader.dataset))

def test_accuracy_NO_DANN(source_dataloader, cartoon_dataloader, sketch_dataloader, lr, step_size):
  
  best_model = {'net': 0, 'acc':0, 'num_epoch': 0}
  net = DANN_alexnet(pretrained=True)

  Log_aux = Log("Inizio stampa dati per: lr = " + str(lr) + " step_size = " +  str(step_size) + ".\n")

  # Define loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

  #Ottimzzo solo i parametri coninvolti nella classificazione senza DANN
  params= [net.features.parameters(), net.classifier.parameters()]

  parameters_to_not_optimize = net.domain_classifier.parameters()
  for p in parameters_to_not_optimize:
      p.requires_grad = False

  # An optimizer updates the weights based on loss
  optimizer = optim.SGD(itertools.chain(*params), lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

  # A scheduler dynamically changes learning rate
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=GAMMA)

  net = net.to(DEVICE)
  cudnn.benchmark # Calling this optimizes runtime
  accuracy_list = []
  
  # Start iterating over the epochs

  for epoch in range(NUM_EPOCH):

    Log_aux.add_epoch([(epoch+1), NUM_EPOCH, scheduler.get_lr()])
    aux_loss = 0
    num_batch = 0

    # Iterate over the dataset
    for images, labels in source_dataloader:

      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      net.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = net(images)

      # Compute loss based on output and ground truth
      loss = criterion(outputs, labels)
      if math.isnan(loss):
        Log_aux.add_loss([loss])
        return  best_model,  Log_aux 

      aux_loss += loss.item()
      num_batch += 1

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients
    
    Log_aux.add_loss([aux_loss / num_batch])

    # Step the scheduler
    scheduler.step() 
    net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

    accuracy_sketch = test(net, sketch_dataloader)
    accuracy_cartoon = test(net, cartoon_dataloader)

    accuracy_avg = (accuracy_cartoon + accuracy_sketch) / 2
    accuracy_avg = round(accuracy_avg * 100, 2)

    if accuracy_avg > best_model['acc']:
      best_model['acc'] = accuracy_avg
      best_model['net'] = copy.deepcopy(net)
      best_model['num_epoch'] = epoch + 1
   
  
  return best_model, Log_aux 

def test_accuracy_DANN(source_dataloader, val_dataloader, lr, step_size, alpha, num_epoch):


  Log_aux = Log("Inizio stampa dati per: lr = " + str(lr) + ", step_size = " +  str(step_size) + ", alpha = " + str(alpha) + ".\n")

  net = DANN_alexnet(pretrained=True)
  # Define loss functions
  criterion_class = nn.CrossEntropyLoss() 
  criterion_domain = nn.CrossEntropyLoss() 

  params = net.parameters()
  optimizer = optim.SGD(params, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=GAMMA)

  net = net.to(DEVICE)
  cudnn.benchmark 
  accuracy_list = []

  for epoch in range(num_epoch):

    Log_aux.add_epoch([epoch+1, num_epoch, scheduler.get_lr()])

    net.train(True) 

    aux_val_dataloader = copy.deepcopy(val_dataloader)
    aux_val_iter_dataloader = iter(aux_val_dataloader)

    aux_loss1 = 0
    aux_loss2 = 0
    num_batch = 0

    # Iterate over the dataset
    for images_source, labels_source in source_dataloader:

      images_val, labels_val = next(aux_val_iter_dataloader)

      images_source = images_source.to(DEVICE)
      labels_source = labels_source.to(DEVICE)
      images_val = images_val.to(DEVICE)


      labels_source_domains = [0]*BATCH_SIZE
      labels_source_domains = torch.LongTensor(labels_source_domains)
      labels_source_domains = labels_source_domains.to(DEVICE)

      labels_val_domains = [1]*BATCH_SIZE
      labels_val_domains = torch.LongTensor(labels_val_domains)
      labels_val_domains = labels_val_domains.to(DEVICE)


      net.train() # Sets module in training mode
      optimizer.zero_grad() # Zero-ing the gradients

      ##################################################################
      #      train on source labels by forwarding source data to Gy    #
      ##################################################################

      # Forward pass to the network
      outputs = net(images_source)

      # Compute loss based on output and ground truth
      loss_classification = criterion_class(outputs, labels_source)
   
      # Calcola il gradiente e lo somma a quelli precedenti
      loss_classification.backward()

      ###################################################################
      #     train the discriminator by forwarding source data to Gd     #
      ###################################################################

      outputs = net(images_source, alpha)

      loss_domain_source = criterion_domain(outputs, labels_source_domains)
      loss_domain_source.backward()  

      ###################################################################
      #     train the discriminator by forwarding val data to Gd     #
      ###################################################################

      outputs = net(images_val, alpha)

      loss_domain_val = criterion_domain(outputs, labels_val_domains)
      loss_domain_val.backward()  


      optimizer.step() # update weights based on accumulated gradients

      aux_loss1 += loss_classification.item()
      aux_loss2 += (loss_domain_source.item() + loss_domain_val.item())/2
      num_batch += 1

    Log_aux.add_loss([aux_loss1 / num_batch, aux_loss2 / num_batch])
    # Step the scheduler
    scheduler.step() 

    # Calcolo accuracy sul val
    accuracy = test(net, val_dataloader)
    accuracy_list.append(round(accuracy*100, 2))

    del aux_val_iter_dataloader

  return accuracy_list, Log_aux


**Prepare Dataloader**

In [0]:
%xmode Verbose
source_domains = ["photo"]
target_domains = ["art_painting"]
cartoon_domains = ["cartoon"]
sketch_domains = ["sketch"]

source_dataloader, target_dataloader, cartoon_dataloader, sketch_dataloader = prepareDataloader(transform, source_domains, target_domains, cartoon_domains, sketch_domains)

Exception reporting mode: Verbose
Art Painting Dataset: 1670
Photo Dataset: 2048
Cartoon Dataset: 2344
Sketch Dataset: 3929


3A

In [0]:
#Hyperparameters for point 3A and 3B
LR = 1e-4
STEP_SIZE = 25
ALPHA = 1

In [0]:

%xmode Verbose
net = DANN_alexnet(pretrained=True)

# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

#Ottimzzo solo i parametri coninvolti nella classificazione senza DANN
params= [net.features.parameters(), net.classifier.parameters()]

parameters_to_not_optimize = net.domain_classifier.parameters()
for p in parameters_to_not_optimize:
    p.requires_grad = False

# An optimizer updates the weights based on loss
optimizer = optim.SGD(itertools.chain(*params), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# A scheduler dynamically changes learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

net = net.to(DEVICE)

cudnn.benchmark # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCH):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCH, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in source_dataloader:

    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('   Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step() 
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

net.train(False) # Set Network to evaluation mode

running_corrects = 0

for images, labels in target_dataloader:
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)


  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()
  

# Calculate Accuracy
accuracy = running_corrects / float(len(target_dataloader.dataset))

print('Test Accuracy: {}%.'.format(round(accuracy*100, 2)))


Exception reporting mode: Verbose
Starting epoch 1/1, LR = [0.0001]
   Step 0, Loss 2.1150975227355957
   Step 5, Loss 1.8422948122024536
Test Accuracy: 16.6%.


3B

In [0]:

%xmode Verbose
net = DANN_alexnet(pretrained=True)

# Define loss function
criterion_class = nn.CrossEntropyLoss() 
criterion_domain = nn.CrossEntropyLoss() 


#Ottimzzo tutti i parametri della rete
params= net.parameters()

# An optimizer updates the weights based on loss
optimizer = optim.SGD(params, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# A scheduler dynamically changes learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

net = net.to(DEVICE)
cudnn.benchmark # Calling this optimizes runtime
current_step = 0

for epoch in range(NUM_EPOCH):

  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCH, scheduler.get_lr()))

  aux_target_dataloader = copy.deepcopy(target_dataloader)
  aux_target_iter_dataloader = iter(aux_target_dataloader)

  # Iterate over the dataset
  for images_source, labels_source in source_dataloader:

    images_target, labels_target = next(aux_target_iter_dataloader)

    images_source = images_source.to(DEVICE)
    labels_source = labels_source.to(DEVICE)
    images_target = images_target.to(DEVICE)
    labels_target = labels_target.to(DEVICE)


    net.train() # Sets module in training mode
    optimizer.zero_grad() # Zero-ing the gradients

    #################
    #     3.B.1     #
    #################

    # Forward pass to the network
    outputs = net(images_source)

    # Compute loss based on output and ground truth
    loss_classification = criterion_class(outputs, labels_source)

  
    # Compute gradients for each layer and update weights
    loss_classification.backward()  # backward pass: computes gradients

    #################
    #     3.B.2     #
    #################

    outputs = net(images_source, ALPHA)

    labels_source = [0]*BATCH_SIZE
    labels_source = torch.LongTensor(labels_source)
    labels_source = labels_source.to(DEVICE)


    loss_photo = criterion_domain(outputs, labels_source)
    loss_photo.backward()  


    #################
    #     3.B.3     #
    #################

    outputs = net(images_target, ALPHA)

    labels_target = [1]*BATCH_SIZE
    labels_target = torch.LongTensor(labels_target)
    labels_target = labels_target.to(DEVICE)

    loss_art = criterion_domain(outputs, labels_target)
    loss_art.backward()  


    optimizer.step() # update weights based on accumulated gradients

  # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print("   Step {", current_step, "}, Loss {", loss_classification.item(), "} on 3.B.1.")
      print("   Step {", current_step, "}, Loss {", loss_photo.item(), "} on 3.B.2.")
      print("   Step {", current_step, "}, Loss {", loss_art.item(), "} on 3.B.3.")
    current_step += 1

  # Step the scheduler
  scheduler.step() 
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda



net.train(False) # Set Network to evaluation mode
running_corrects = 0

for images, labels in target_dataloader:

  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)


  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()
  

# Calculate Accuracy
accuracy = running_corrects / float(len(target_dataloader.dataset))

print('Test Accuracy: {}%.'.format(round(accuracy*100, 2)))


Exception reporting mode: Verbose
Starting epoch 1/1, LR = [0.0001]
   Step { 0 }, Loss { 2.3229129314422607 } on 3.B.1.
   Step { 0 }, Loss { 0.7061945796012878 } on 3.B.2.
   Step { 0 }, Loss { 0.7256376147270203 } on 3.B.3.
   Step { 5 }, Loss { 1.9416784048080444 } on 3.B.1.
   Step { 5 }, Loss { 0.7526364922523499 } on 3.B.2.
   Step { 5 }, Loss { 0.6706061959266663 } on 3.B.3.
Test Accuracy: 15.48%.


**4.A e 4.B**

In [0]:
#Hyperparameter for point 4A-B
STEP_SIZES = [25, 30]    
LRS= [1e-2, 5e-3, 1e-3] 

In [0]:
%xmode Verbose

top_model = { 'lr':0, 'acc': 0, 'step_size': 0, 'num_epoch': 0}
Log_1 = Log("")

for lr in LRS:
  for step_size in STEP_SIZES:

    print("------------------------------------------------------------------------")
    print("Starting ( LR =", lr,  ", Step size =", step_size, ").")

    best_model_test, Log_aux = test_accuracy_NO_DANN(source_dataloader, sketch_dataloader, cartoon_dataloader, lr, step_size)   
    Log_1.concLOG(Log_aux)    

    print("Ending ( LR =", lr,  ", Step size =", step_size, "Num epoch =", best_model_test['num_epoch'], ").")
    print("Accuratezza media migliore ", best_model_test['acc'], ".")

    if top_model['acc'] < best_model_test['acc'] :
      top_model['lr'] = lr
      top_model['acc'] = best_model_test['acc']
      top_model['step_size'] = step_size
      top_model['num_epoch'] = best_model_test['num_epoch']
      top_model['net'] = best_model_test['net']

print("------------------------------------------------------------------------")
print("Iperparametri migliori: lr=", top_model['lr'], ", step_size=", top_model['step_size'], ". Accurattezza massima =", top_model['acc'], ". Numero di epoche:", top_model['num_epoch'])

print("\n\n-------------------> Accuracy sul target domain:", str(round(test(top_model['net'], target_dataloader)*100, 2)) + "%.")

#Log_1.print_log()
#Log_1.print_loss()


Exception reporting mode: Verbose
------------------------------------------------------------------------
Starting ( LR = 0.01 , Step size = 25 ).
Ending ( LR = 0.01 , Step size = 25 Num epoch = 1 ).
Accuratezza media migliore  19.75 .
------------------------------------------------------------------------
Starting ( LR = 0.01 , Step size = 30 ).
Ending ( LR = 0.01 , Step size = 30 Num epoch = 1 ).
Accuratezza media migliore  22.47 .
------------------------------------------------------------------------
Starting ( LR = 0.005 , Step size = 25 ).
Ending ( LR = 0.005 , Step size = 25 Num epoch = 1 ).
Accuratezza media migliore  27.3 .
------------------------------------------------------------------------
Starting ( LR = 0.005 , Step size = 30 ).
Ending ( LR = 0.005 , Step size = 30 Num epoch = 1 ).
Accuratezza media migliore  23.47 .
------------------------------------------------------------------------
Starting ( LR = 0.001 , Step size = 25 ).
Ending ( LR = 0.001 , Step size = 25

In [0]:
#Hyperparameters for point 4C-D
LRS= [1e-3, 5e-4, 1e-4]           
STEP_SIZES = [25, 30]       
ALPHAS = [0.5, 0.3, 0.1]        

**4.C e 4.D**

In [0]:
%xmode Verbose

top_model = { 'lr':0, 'acc': 0, 'step_size': 0, 'num_epoch': 0, 'alpha': 0}
Log_1 = Log("")

for lr in LRS:
  for step_size in STEP_SIZES:
    for alpha in ALPHAS:

      print("------------------------------------------------------------------------")
      print("Starting ( LR =", lr,  ", Step size =", step_size, ", Alpha = ", alpha, ").")

      accuracy_list_sketch, log_aux1 = test_accuracy_DANN(source_dataloader, sketch_dataloader, lr, step_size, alpha, NUM_EPOCH)      
      accuracy_list_cartoon, log_aux2 = test_accuracy_DANN(source_dataloader, cartoon_dataloader, lr, step_size, alpha, NUM_EPOCH)

      Log_1.concLOG(log_aux1)
      Log_1.concLOG(log_aux2)

      accuracy_list_avg = []
      for index in range(len(accuracy_list_sketch)):
        accuracy_list_avg.append((accuracy_list_sketch[index] + accuracy_list_cartoon[index])/2)

      accuracy_avg_max = max(accuracy_list_avg)
      num_epoch = accuracy_list_avg.index(accuracy_avg_max) + 1

      print("Ending ( LR =", lr,  ", Step size =", step_size,"Alpha =", alpha,  "num epoch =", num_epoch, ").")
      print("Accuratezza media massima:", accuracy_avg_max, ".")

      if top_model['acc'] < accuracy_avg_max :
        top_model['lr'] = lr
        top_model['acc'] = accuracy_avg_max
        top_model['step_size'] = step_size
        top_model['num_epoch'] = num_epoch
        top_model['alpha'] = alpha

print("------------------------------------------------------------------------")
print("Iperparametri migliori: lr=", top_model['lr'], ", step_size=", top_model['step_size'], ". Accurattezza massima =", top_model['acc'], ". Numero di epoche:", top_model['num_epoch'])

accuracy_target = test_accuracy_DANN(source_dataloader, target_dataloader, top_model['lr'], top_model['step_size'],top_model['alpha'], top_model['num_epoch'])
print(" --------------------------------> Accuracy sul target domain:", accuracy_target[0][len(accuracy_target[0]) - 1])

#Log_1.print_log()
Log_1.print_loss()
  

Exception reporting mode: Verbose
------------------------------------------------------------------------
Starting ( LR = 0.001 , Step size = 25 , Alpha =  0.5 ).
Ending ( LR = 0.001 , Step size = 25 Alpha = 0.5 num epoch = 21 ).
Accuratezza media massima: 37.99 .
------------------------------------------------------------------------
Starting ( LR = 0.001 , Step size = 25 , Alpha =  0.3 ).
Ending ( LR = 0.001 , Step size = 25 Alpha = 0.3 num epoch = 26 ).
Accuratezza media massima: 33.595 .
------------------------------------------------------------------------
Starting ( LR = 0.001 , Step size = 25 , Alpha =  0.1 ).
Ending ( LR = 0.001 , Step size = 25 Alpha = 0.1 num epoch = 18 ).
Accuratezza media massima: 28.634999999999998 .
------------------------------------------------------------------------
Starting ( LR = 0.001 , Step size = 30 , Alpha =  0.5 ).
Ending ( LR = 0.001 , Step size = 30 Alpha = 0.5 num epoch = 18 ).
Accuratezza media massima: 37.405 .
-----------------------